# Aplicación del moelo de 496 clases a datos con un nodo apagos pre y post normalización

### Librerias y funciones

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio
import pandas as pd
import itertools
from collections import Counter
import matplotlib.pyplot as plt

### Carga de datos

In [ ]:
mat = sio.loadmat('Spectrums_removingNode_afterNormalization.mat') #Se debe establecer la ruta del archivo que se desee usar


nodo = []
espectro = []

# Procesamos los datos
for i in range(len(mat['Spectrums'])):
  for j in mat['Spectrums'][i]:
    nodo.append(mat['nodes'][0][i])
    espectro.append(j)

dicta = {'nodos': nodo, 'Espectro': espectro}
df = pd.DataFrame(dicta) # Creamos el DataFrame

df['Cortada'] = df['Espectro'].apply(lambda x: np.array(x[0:500])) # Limitamos la entrada a una longitud de 500
print(df.head())


x_test = np.array(list(df['Cortada']))
nodos = np.array(list(df['nodos']))

### Carga del modelo

Elegimos el modelo que deseamos.

In [ ]:
#model = keras.models.load_model("/home/carmen/datos/TFG_carpeta_modelo/mejor_modelo_4clases_1/") # Modelo 4 clases
#model= keras.models.load_model("/home/carmen/datos/TFG_carpeta_modelo/mejor_modelo_121clases_2/") # Modelo 121 clases
#model = keras.models.load_model("/home/carmen/datos/TFG_carpeta_modelo/mejor_modelo3/") # Modelo 256 clases
model = keras.models.load_model("/home/carmen/datos/TFG_carpeta_modelo/mejor_modelo_496clases_4/") # Modelo 496 clases

### Calculamos la predicción 

In [ ]:
p4 = []
for i in range(len(x_test)):
    p4.append(np.argmax(model.predict(x_test[i:(i+1)], verbose= 0)))

In [ ]:
p14 = p4[:90] # Predicción nodo 14
p17 = p4[90:180] # Predicción nodo 17
p76 = p4[180:] # Predicción nodo 76


### Calcular tau y kappa

In [ ]:
# Definimos los rangos de valores para K y Tau y creamos un DataFrame para saber la equivalencia entre clases y variables
K_values = range(0, 16)  # de 0 a 15
Tau_values = range(0, 31)  # de 0 a 30
combinations = list(itertools.product(K_values, Tau_values))
df = pd.DataFrame(combinations, columns=['K', 'Tau'])

In [ ]:
# Resultado de las predicciones. En este caso se utiliza las predicciones cuando el nodo 76 está apagado. 
# Para ver el resultado de las otras predicciones solo tendríamos que cambiar p76 por 'p14' o 'p17'

conteo = Counter(p76) 
conteo

In [ ]:

k_p =[]
tau_p=[]

for i in conteo.keys(): #las clases predichas
    print(i, df.iloc[i][0], df.iloc[i][1])
    
    k_p.append(df.iloc[i][0]) #K predicha
    tau_p.append(df.iloc[i][1]) #tau predicha

In [ ]:

n_p = conteo.values() #predicciones para cada clase

size = []
leg = []
s_leyenda = []

for i in n_p:
    size.append(i*10) #representar
    s_leyenda.append(i)
    
    
sorted_indices = np.argsort(size)[::-1]
k_p = np.array(k_p)
tau_p = np.array(tau_p)
size = np.array(size)
s_leyenda = np.array(s_leyenda)

s_leyenda = s_leyenda[sorted_indices]
k_p = k_p[sorted_indices]
tau_p = tau_p[sorted_indices]

size = size[sorted_indices] 

k_leyenda = k_p
tau_leyenda = tau_p

size= size.tolist()
tau_p = tau_p.tolist()
k_p =k_p.tolist()

bueno = size.pop(0) 
x_bueno = tau_p.pop(0)  
y_bueno = k_p.pop(0)



### Representación de los resultados

In [ ]:
x_values = tau_p  # Valores del eje X de 0 a 100
y_values = k_p  # Valores del eje Y en la diagonal proporcional
sizes = size 


highlight_point = (21, 4)  # Resaltamos la clase con los valores usados en la simulación


plt.figure(figsize=(10, 6))
plt.scatter(x_values, y_values, color='blue',s = sizes)
plt.scatter(*highlight_point, color='lightgreen', s=bueno)

plt.yticks(range(0,16,1))
plt.xticks(range(0,31,1))
plt.title('Predicciones con nodo 76 apagado después de la normalización, 496 clases')
plt.xlabel('Tau')
plt.ylabel('K')
handles = [
    plt.Line2D([], [], marker='o', color='blue', markersize=10, linestyle='None', label='Otras predicciones'),
    plt.Line2D([], [], marker='o', color='lightgreen', markersize=10, linestyle='None', label='Correcta')
]
labels = ['Otras predicciones', 'Predicción correcta']

# Añadimos la leyenda 
plt.legend(handles=handles, labels=labels)
plt.grid(True)
ax2 = plt.twinx()
ax2.grid(False)
ax2.set_yticks([])

# Texto de las predicciones
textos = [f'Tau = {x}, K = {y}, Número de predicciones = {z}' for x, y, z in zip(tau_l, kl_l, leg)]

for i, texto in enumerate(textos):
    color = 'green' if i == 0 else 'black'  
    ax2.text(1.02, 1 - i / len(textos), texto, transform=ax2.transAxes,
             ha='left', va='top', fontsize=10, color=color)


plt.subplots_adjust(right=0.95)
plt.show()
